<a href="https://colab.research.google.com/github/yura-eng/G4/blob/main/cloze_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**📣Cloze test with TTS 📣**

In [ ]:
!pip install gradio gtts pydub

import gradio as gr
import requests
from gtts import gTTS
from pydub import AudioSegment
from pydub.generators import Sine
import os

# Cloze questions based on the given text with past tense verbs
cloze_questions = [
    {
        "question": "A Fox one day [BLANK] a beautiful bunch of ripe grapes hanging from a vine trained along the branches of a tree.",
        "answer": "spied",
        "hint": "spy"
    },
    {
        "question": "The grapes seemed ready to burst with juice, and the Fox's mouth watered as he [BLANK] longingly at them.",
        "answer": "gazed",
        "hint": "gaze"
    },
    {
        "question": "The bunch [BLANK] from a high branch, and the Fox had to jump for it.",
        "answer": "hung",
        "hint": "hang"
    },
     {
        "question": "The first time he [BLANK] he missed it by a long way.",
        "answer": "jumped",
        "hint": "jump"
    },
    {
        "question": "So he walked off a short distance and [BLANK] a running leap at it, only to fall short once more.",
        "answer": "took",
        "hint": "take"
    },
    {
        "question": "Again and again he [BLANK], but in vain.",
        "answer": "tried",
        "hint": "try"
    },
    {
        "question": "Now he [BLANK] down and looked at the grapes in disgust.",
        "answer": "sat",
        "hint": "sit"
    },
    {
        "question": 'What a fool I am," he [BLANK]. "Here I am wearing myself out to get a bunch of sour grapes that are not worth gaping for.',
        "answer": "said",
        "hint": "say"
    },
    {
        "question": "And off he [BLANK] very, very scornfully.",
        "answer": "walked",
        "hint": "walk"
    }
]

# Function to create a bell sound
def create_bell_sound(filename="bell.wav"):
    # Generate a bell sound (1000 Hz tone for 200 ms)
    tone = Sine(1000).to_audio_segment(duration=200).apply_gain(-10).fade_in(50).fade_out(50)
    tone.export(filename, format="wav")

# Create the bell sound file
create_bell_sound()

# Function to convert text to speech and add bell sound
def text_to_speech_with_bell(text, filename):
    tts = gTTS(text.replace("[BLANK]", ""))
    tts.save("temp.mp3")

    # Load the generated speech and bell sound
    speech = AudioSegment.from_mp3("temp.mp3")
    bell = AudioSegment.from_wav("bell.wav")

    # Find the position of the blank and insert the bell sound
    blank_position = text.find("[BLANK]")
    part1 = text[:blank_position]
    part2 = text[blank_position + len("[BLANK]"):]

    tts_part1 = gTTS(part1)
    tts_part1.save("part1.mp3")
    tts_part2 = gTTS(part2)
    tts_part2.save("part2.mp3")

    speech_part1 = AudioSegment.from_mp3("part1.mp3")
    speech_part2 = AudioSegment.from_mp3("part2.mp3")

    # Create a silent segment (2 seconds)
    silent_segment = AudioSegment.silent(duration=2000)

    combined = speech_part1 + bell + silent_segment + speech_part2

    # Save the final audio with the bell sound inserted
    combined.export(filename, format="mp3")
    os.remove("temp.mp3")
    os.remove("part1.mp3")
    os.remove("part2.mp3")
    return filename

# Function to convert text to speech without any modifications
def text_to_speech(text, filename):
    tts = gTTS(text)
    tts.save(filename)
    return filename

# Generate audio files for questions
for i, question in enumerate(cloze_questions):
    full_audio_filename = f"full_question_{i+1}.mp3"
    cloze_audio_filename = f"cloze_question_{i+1}.mp3"
    question["full_audio"] = text_to_speech(question["question"].replace("[BLANK]", question["answer"]), full_audio_filename)
    question["cloze_audio"] = text_to_speech_with_bell(question["question"], cloze_audio_filename)

# Function to handle the cloze quiz
def cloze_quiz(state, answer):
    name, question_index, score, results = state
    question = cloze_questions[question_index]

    correct = answer.strip().lower() == question["answer"].lower()

    if correct:
        score += 1
        results.append(f"Question {question_index + 1}: Correct🎉\n")
    else:
        results.append(f"Question {question_index + 1}: Incorrect🚩, the correct answer is: {question['answer']}\n")

    question_index += 1

    if question_index < len(cloze_questions):
        next_full_audio = cloze_questions[question_index]["full_audio"]
        next_cloze_audio = cloze_questions[question_index]["cloze_audio"]
        next_hint = f"Hint: {cloze_questions[question_index]['hint']} ◁◁◁ Check out this verb!"
        return (name, question_index, score, results), next_full_audio, next_cloze_audio, next_hint, gr.update(visible=False), gr.update(value="", interactive=True, visible=True)
    else:
        result_text = f"* Name: {name}\n* Score: {score} out of {len(cloze_questions)}\n" + "\n".join(results)
        return (name, question_index, score, results), None, None, "", gr.update(visible=True, value=result_text), gr.update(visible=False)

# Function to start the quiz
def start_quiz(name):
    hint = f"Hint: {cloze_questions[0]['hint']} ◁◁◁ Check out this verb!"
    return (name, 0, 0, []), cloze_questions[0]["full_audio"], cloze_questions[0]["cloze_audio"], hint, gr.update(visible=False), gr.update(visible=True)

# Create the Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# Listening Cloze Test Instructions")
    gr.Markdown("""
    **Instructions:**

    1. Listen to the full sentence.
    2. Listen to the cloze question with the beep sound.
    3. Write the correct form of the verb you hear in the blank space. Remember, you do not need to write the entire sentence, just the verb.

    **For example, if you hear "Yesterday, He went to the sea," and then you hear "Yesterday, He ___ to the sea" with a beep sound, you should write "went" if that is the verb you heard.**

    **Are you ready? Let's begin!**
    """)

    name_input = gr.Textbox(label="Enter your name")
    start_button = gr.Button("🌟Start Quiz🌟")
    full_question_audio = gr.Audio(interactive=True, autoplay=False, label="Full Sentence Listening")
    cloze_question_audio = gr.Audio(interactive=True, autoplay=False, label="Cloze Question")
    hint_output = gr.Markdown()
    answer_input = gr.Textbox(label="Your Answer📝")
    next_button = gr.Button("Next👉")
    result_output = gr.Textbox(label="Results", interactive=False, visible=False)

    # Initialize the state
    state = gr.State()

    start_button.click(start_quiz, inputs=name_input, outputs=[state, full_question_audio, cloze_question_audio, hint_output, result_output, answer_input])
    next_button.click(cloze_quiz, inputs=[state, answer_input], outputs=[state, full_question_audio, cloze_question_audio, hint_output, result_output, answer_input])

iface.launch(share=True)